<a href="https://colab.research.google.com/github/AFMac/MLZoomCamp/blob/main/mod8_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
import os
from torch.utils.data import Dataset
from PIL import Image

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
from torchvision import transforms

input_size = 200

# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [7]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

val_dataset = HairDataset(
    data_dir='./data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [27]:
import torch.nn as nn
import torchvision.models as models

class HairstyleClassifierMobileNet(nn.Module):
    def __init__(self, num_classes=2):
        super(HairstyleClassifierMobileNet, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=(3,3),
            padding=0,
            stride=1
        )
        self.maxpool = nn.MaxPool2d(
            kernel_size=(2,2),
        )
        self.linear1 = nn.Linear(313632, 64)
        self.relu_linear = nn.ReLU()
        self.linear2 = nn.Linear(64, 1)
        #self.output_activation = nn.Sigmoid()


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x,1)
        x = self.linear1(x)
        x = self.relu_linear(x)
        x = self.linear2(x)
        #x = self.output_activation(x)
        return x

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HairstyleClassifierMobileNet()
model.to(device)

HairstyleClassifierMobileNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=313632, out_features=64, bias=True)
  (relu_linear): ReLU()
  (linear2): Linear(in_features=64, out_features=1, bias=True)
)

In [29]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum = 0.8)

In [30]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
              ReLU-4                   [-1, 64]               0
            Linear-5                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.97
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [31]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [32]:
train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

val_dataset = HairDataset(
    data_dir='./data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

In [33]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6280, Acc: 0.6479, Val Loss: 0.6328, Val Acc: 0.6119
Epoch 2/10, Loss: 0.5496, Acc: 0.7216, Val Loss: 0.5855, Val Acc: 0.6866
Epoch 3/10, Loss: 0.4856, Acc: 0.7516, Val Loss: 0.5934, Val Acc: 0.6716
Epoch 4/10, Loss: 0.4379, Acc: 0.7915, Val Loss: 0.5791, Val Acc: 0.6965
Epoch 5/10, Loss: 0.4159, Acc: 0.8002, Val Loss: 0.5889, Val Acc: 0.6816
Epoch 6/10, Loss: 0.2959, Acc: 0.8814, Val Loss: 0.7116, Val Acc: 0.7015
Epoch 7/10, Loss: 0.2553, Acc: 0.8951, Val Loss: 0.6756, Val Acc: 0.7313
Epoch 8/10, Loss: 0.1797, Acc: 0.9338, Val Loss: 0.7353, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2303, Acc: 0.9126, Val Loss: 0.7110, Val Acc: 0.7164
Epoch 10/10, Loss: 0.1180, Acc: 0.9563, Val Loss: 0.6520, Val Acc: 0.7413


In [37]:
import statistics
data = [.6479, .7216, .7516, .7915, .8002, .8814, .8951, .9338, .9126, .9563]
print(statistics.median(data))

0.8408


In [38]:
data = [.6280, .5496, .4856, .4379, .4159, .2959, .2553, .1797, .2303, .1180]
print(statistics.stdev(data))

0.16855740070769165


In [41]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])



In [42]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.0687, Acc: 0.9888, Val Loss: 0.9319, Val Acc: 0.7214
Epoch 2/10, Loss: 0.1386, Acc: 0.9538, Val Loss: 0.7683, Val Acc: 0.7015
Epoch 3/10, Loss: 0.0345, Acc: 0.9975, Val Loss: 0.8174, Val Acc: 0.7065
Epoch 4/10, Loss: 0.0179, Acc: 0.9988, Val Loss: 0.9301, Val Acc: 0.7214
Epoch 5/10, Loss: 0.0136, Acc: 1.0000, Val Loss: 0.8674, Val Acc: 0.7065
Epoch 6/10, Loss: 0.0090, Acc: 1.0000, Val Loss: 0.8833, Val Acc: 0.7313
Epoch 7/10, Loss: 0.0072, Acc: 1.0000, Val Loss: 0.9118, Val Acc: 0.7065
Epoch 8/10, Loss: 0.0060, Acc: 1.0000, Val Loss: 0.9280, Val Acc: 0.7065
Epoch 9/10, Loss: 0.0055, Acc: 1.0000, Val Loss: 0.9615, Val Acc: 0.7065
Epoch 10/10, Loss: 0.0044, Acc: 1.0000, Val Loss: 0.9606, Val Acc: 0.7264


In [43]:
data = [.9319, .7683, .8174, .9301, .8674, .8833, .9118, .9280, .9615, .9606]
print(statistics.mean(data))

0.89603


In [44]:
data = [.7264, .7065, .7065, .7065, .7313]
print(statistics.mean(data))

0.71544
